In [1]:
import numpy as np
import pandas as pd
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import csv

[nltk_data] Downloading package stopwords to /Users/bala/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/bala/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#path = r"C:\Users\hari\Downloads"
#train_data = pd.read_table(path+r"\1643662645_9617953_1567602457_126649_test (2).dat")
#test_data = pd.read_table(path+path+r"\1643662645_8986752_1567602457_1187546_train_file (3).dat")

import os
path = "/Users/harivigneshgomathi/Downloads/"
dir_list = os.listdir(path)
train_data = pd.read_csv(path+"1643662645_8986752_1567602457_1187546_train_file.dat", sep='\t', header=None)

In [3]:
datContent = [i.strip() for i in open(path+"1643662645_9617953_1567602457_126649_test.dat").readlines()]
dictionary = {'data': datContent}  
test_data = pd.DataFrame(dictionary) 


In [4]:
#merging both the train and test dataset to do preprocessing
train_datas = pd.DataFrame(train_data.iloc[:,1]) 
test_datas = pd.DataFrame(test_data.iloc[:,0]) 
total_datas = pd.concat([train_datas.iloc[:,0], test_data.iloc[:,0]],ignore_index = True)

In [5]:
#Preprocessing

words = stopwords.words("english")
#Stem the train file
final = ''
stemmer = SnowballStemmer('english')

#remove stopwords
for i in range(len(total_datas)):
    line = total_datas.iloc[i]
    temp = ''
    for word in str(line).split():
        if word not in words:
            temp = temp +" "+word
    total_datas.iloc[i] = temp
    
#stem words
for i in range(len(total_datas)):
    line = total_datas.iloc[i]
    temp = ''
    for word in str(line).split():
        temp = temp +" "+ stemmer.stem(word)
    total_datas.iloc[i] = temp


In [7]:
#vectorize
vectorizer = CountVectorizer()
transformed_total_data = vectorizer.fit_transform(total_datas.iloc[:]).toarray()
vectorizer.get_feature_names()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '000',
 '000importer',
 '000w',
 '002',
 '00684',
 '00am',
 '00pm',
 '01',
 '0186010',
 '02',
 '03',
 '04',
 '0409',
 '0442fax',
 '05',
 '0543',
 '06',
 '06at',
 '06mi',
 '07',
 '07060c',
 '08',
 '09',
 '092304983',
 '0ghz',
 '0s',
 '10',
 '100',
 '1000',
 '10000',
 '1000000',
 '10000000',
 '1000sf',
 '1000x',
 '1001',
 '1002',
 '100ct',
 '100f',
 '100ft',
 '100lbs',
 '100ml',
 '100th',
 '100x',
 '101',
 '102',
 '103',
 '104',
 '10409',
 '10409a',
 '105',
 '105lbs',
 '106',
 '106lbs',
 '107lbs',
 '108',
 '1080p',
 '109',
 '10am',
 '10day',
 '10hrs',
 '10in',
 '10ish',
 '10lb',
 '10lbs',
 '10m',
 '10min',
 '10minut',
 '10mo',
 '10month',
 '10mos',
 '10mph',
 '10mts',
 '10oz',
 '10pack',
 '10pm',
 '10pound',
 '10stack',
 '10th',
 '10wk',
 '10x',
 '10xs',
 '11',
 '110',
 '1100',
 '1100watt',
 '110lb',
 '110lbs',
 '110v',
 '111',
 '112',
 '1124937150',
 '11296',
 '114',
 '115',
 '115lbs',
 '116',
 '1160',
 '117',
 '118',
 '1185',
 '119',
 '119lbs',
 '11g',
 '11i',
 '11lb',
 '11lbs'

In [9]:
#splitting the total datas into training and testing
train_data_transformed = pd.DataFrame(transformed_total_data[0:18506])
test_data_transformed = pd.DataFrame(transformed_total_data[18506:])

In [17]:
#training the classifier
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression()
clf_lr.fit(train_data_transformed, train_data.iloc[:,0])

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [18]:
y_pred = clf_lr.predict(test_data_transformed)

[ 1  1  1 ... -1 -1 -1]


In [21]:
# to save the output predictions as a numpy file
np.save(path+'output_predictions.npy', y_pred)

# to save the output predictions as a text file
a_file = open(path+"output_predictions.txt", "w")
np.savetxt(a_file, y_pred)
a_file.close()

#to save the output predictions as a data file
predictions = pd.DataFrame(y_pred)
predictions.to_csv(path+"output_predictions.dat", index=False, header=None)